In [35]:
import numpy as np
import pandas
import warnings
warnings.filterwarnings("ignore")

from keras.datasets import mnist
import keras.backend as K
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dense, Input, Lambda, Activation, Flatten
from keras.models import Model
from keras.regularizers import l2

In [36]:
import cv2

In [40]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.utils import shuffle

## Constant variable

In [9]:
BATCH_SIZE = 64
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHANNEL = 1

## Model 

In [6]:
K.image_data_format()

'channels_last'

In [24]:
class ALexNet:
    def __init__(self, img_width, img_height, img_channel, reg=1e-4):
        self.img_height = img_height
        self.img_width = img_width
        self.img_channel = img_channel
        self.reg = reg
        self.channel_axis = -1
        self.data_format = "channels_last"
        
        if K.image_data_format() == "channels_first":
            self.channel_axis = 0
            self.data_format = "channels_first"
#         super(self, ALexNet).__init__(img_width, img_height, img_channel)
        
    def build(self):
        input_shape = (self.img_height, self.img_width, self.img_channel)
        inputs = Input(shape=input_shape, name="input")
        
        x = Conv2D(96, kernel_size=(11, 11), strides=(4, 4), padding="same", kernel_regularizer=l2(self.reg))(inputs)
        x = BatchNormalization(axis=self.channel_axis)(x)
        x = Activation("relu")(x)
        x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
        
        x = Conv2D(256, kernel_size=(5, 5), strides=(1, 1), padding="same", kernel_regularizer=l2(self.reg))(x)
        x = BatchNormalization(axis=self.channel_axis)(x)
        x = Activation("relu")(x)
        x = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(x)
        
        x = Conv2D(384, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_regularizer=l2(self.reg))(x)
        x = BatchNormalization(axis=self.channel_axis)(x)
        x = Activation("relu")(x)
    
        x = Conv2D(384, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_regularizer=l2(self.reg))(x)
        x = BatchNormalization(axis=self.channel_axis)(x)
        x = Activation("relu")(x)
        
        x = Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding="same", kernel_regularizer=l2(self.reg))(x)
        x = BatchNormalization(axis=self.channel_axis)(x)
        x = Activation("relu")(x)
        
        x = Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding="same", kernel_regularizer=l2(self.reg))(x)
        x = BatchNormalization(axis=self.channel_axis)(x)
        x = Activation("relu")(x)
        
        x = Flatten()(x)
        x = Dense(9216, kernel_regularizer=l2(self.reg))(x)
        x = Activation("relu")(x)
        x = Dense(4096, kernel_regularizer=l2(self.reg))(x)
        x = Activation("relu")(x)
        x = Dense(4096, kernel_regularizer=l2(self.reg))(x)
        x = Activation("relu")(x)
        x = Dense(1000, kernel_regularizer=l2(self.reg))(x)
        x = Activation("relu")(x)
        x = Dense(10)(x)
        x = Activation("softmax")(x)
        
        model = Model(inputs=inputs, outputs=x)
        
        return model

In [25]:
alexnet = ALexNet(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL)

In [26]:
model = alexnet.build()

In [27]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 96)          11712     
_________________________________________________________________
batch_normalization_12 (Batc (None, 7, 7, 96)          384       
_________________________________________________________________
activation_14 (Activation)   (None, 7, 7, 96)          0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 3, 3, 96)          0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 3, 256)         614656    
_________________________________________________________________
batch_normalization_13 (Batc (None, 3, 3, 256)         1024  

## Data preparation

In [31]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [33]:
y_train.shape

(60000,)

In [42]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [43]:
lbl = LabelBinarizer()

In [44]:
lbl.fit(y_train)

LabelBinarizer()

In [45]:
y_train = lbl.transform(y_train)

In [46]:
y_test = lbl.transform(y_test)

In [48]:
lbl.classes_

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [49]:
X_train, X_test = np.expand_dims(X_train, axis=-1), np.expand_dims(X_test, axis=-1)

In [41]:
class DataGeneration:
    def __init__(self,
                 X_train, y_train,
                 X_test, y_test,
                 img_height=IMG_HEIGHT, 
                 img_width=IMG_WIDTH, 
                 img_channel=IMG_CHANNEL,
                 batch_size=BATCH_SIZE):
        self.img_height = img_height
        self.img_width = img_width
        self.img_channel = img_channel
        self.batch_size = batch_size
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.current_train = 0
        self.current_test = 0
    
    def load_batch(self, task="train"):
        self.current_index = self.current_train
        self.max_length = self.X_train.shape[0]
        self.X = self.X_train
        self.y = self.y_train
        
        if task != "train":
            self.current_index = self.current_test
            self.max_length = self.X_test.shape[0]
            self.X = self.X_test
            self.y = self.y_test
        
        if (self.current_index + self.batch_size > self.max_length):
            self.current_index = 0
            
            if task != "train":
                self.current_test = 0
            else:
                self.current_train = 0
            
            self.X, self.y = shuffle(self.X, self.y, random_state=42)
        
        batch_X = self.X[self.current_index:self.current_index+self.batch_size]
        batch_y = self.y[self.current_index:self.current_index+self.batch_size]
        
        if task != "train":
            self.current_test += self.batch_size
        else:
            self.current_train += self.batch_size
            
        return batch_X, batch_y
    
    def generator(self, task="train"):
        while True:
            batch_X, batch_y = self.load_batch(task=task)
            
            yield (batch_X, batch_y)

In [50]:
data_generator = DataGeneration(X_train, y_train, X_test, y_test)

## Training model

In [53]:
from keras.optimizers import Adam

In [78]:
opt = Adam(learning_rate=1e-3, decay=1e-4)

In [79]:
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [80]:
model

In [82]:
model.fit_generator(data_generator.generator(),
                    steps_per_epoch=data_generator.X_train.shape[0] // data_generator.batch_size,
                    validation_data=data_generator.generator(task="val"),
                    validation_steps=data_generator.X_test.shape[0] // data_generator.batch_size,
                    epochs=10,
                    initial_epoch=0,
                    verbose=1
                   )

Epoch 1/10
 15/937 [..............................] - ETA: 14:24 - loss: 1.5331 - accuracy: 0.4875

KeyboardInterrupt: 